In [3]:
!pip install mistralai pandas pyarrow dotenv #wget

   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.8 MB 640.0 kB/s eta 0:00:41
   ---------------------------------------- 0.2/25.8 MB 1.7 MB/s eta 0:00:15
    --------------------------------------- 0.4/25.8 MB 3.1 MB/s eta 0:00:09
    --------------------------------------- 0.6/25.8 MB 3.7 MB/s eta 0:00:07
   - -------------------------------------- 0.9/25.8 MB 4.0 MB/s eta 0:00:07
   - -------------------------------------- 1.0/25.8 MB 4.1 MB/s eta 0:00:07
   - -------------------------------------- 1.2/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.4/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.8/25.8 MB 4.0 MB/s eta 0:00:06
   --- ------------------------------------ 2.1/25.8 MB 4.2 MB/s eta 0:00:06
   --- ----

In [17]:
## qua ci vorra il dataset dei pokemon
# TODO FRA: fai una prova con https://huggingface.co/datasets/mii-community/ultrafeedback-translated-ita  oppure cognitivecomputations/ultrachat-uncensored

import pandas as pd
df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [7]:
# controllare che ho file tipo ultrachat_chunk_train.jsonl e ultrachat_chunk_eval.jsonl

In [11]:
# ## script per controllare il formato
# # download the validation and reformat script
# import wget
# url = 'https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py'
# wget.download(url)

'reformat_data.py'

In [21]:
# validate and reformat the training data
!python reformat_data.py ultrachat_chunk_train.jsonl

Skipped 3674th sample
Skipped 9176th sample
Skipped 10559th sample
Skipped 13293th sample
Skipped 13973th sample
Skipped 15219th sample


In [20]:
# validate the reformat the eval data
!python reformat_data.py ultrachat_chunk_eval.jsonl

In [19]:
# solo per osservare il dataset
df_train.iloc[3674]['messages']

array([{'content': 'What are the dimensions of the cavity, product, and shipping box of the Sharp SMC1662DS microwave?: With innovative features like preset controls, Sensor Cooking and the Carousel® turntable system, the Sharp® SMC1662DS 1.6 cu. Ft. Stainless Steel Carousel Countertop Microwave makes reheating your favorite foods, snacks and beverages easier than ever. Use popcorn and beverage settings for one-touch cooking. Express Cook allows one-touch cooking up to six minutes. The convenient and flexible "+30 Sec" key works as both instant start option and allows you to add more time during cooking.\nThe Sharp SMC1662DS microwave is a bold design statement in any kitchen. The elegant, grey interior and bright white, LED interior lighting complements the stainless steel finish of this premium appliance.\nCavity Dimensions (w x h x d): 15.5" x 10.2" x 17.1"\nProduct Dimensions (w x h x d): 21.8" x 12.8" x 17.7"\nShipping Dimensions (w x h x d) : 24.4" x 15.0" x 20.5"', 'role': 'user

In [34]:
# Upload dataset to Mistral
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient

load_dotenv()
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("ultrachat_chunk_train_reformatted.jsonl", "rb") as f:
    ultrachat_chunk_train = client.files.create(file=("ultrachat_chunk_train.jsonl", f))
with open("ultrachat_chunk_eval_reformatted.jsonl", "rb") as f:
    ultrachat_chunk_eval = client.files.create(file=("ultrachat_chunk_eval.jsonl", f))

MistralAPIException: Status: 422. Message: {"detail": "Invalid file format.", "description": "Found at least 538 errors in this file. You can view supported formats here: https://docs.mistral.ai/capabilities/finetuning. ", "errors": [{"message": "Expected last role Assistant for finetuning but got user", "line_number": 1}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 2}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 3}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 4}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 5}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 6}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 7}, {"message": "Expected last role Assistant for finetuning but got user", "line_number": 8}, {"message": "Expected last role Assist

In [ ]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [ ]:
pprint(ultrachat_chunk_train)

In [ ]:
pprint(ultrachat_chunk_eval)

In [ ]:
from mistralai.models.jobs import TrainingParameters
from mistralai.models.jobs import WandbIntegrationIn

WANDB_API_KEY = os.environ.get("WANDB_API_KEY")

created_jobs = client.jobs.create(
    model= "mistral-small-latest", #"open-mistral-7b",
    training_files=[ultrachat_chunk_train.id],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001, ##TODO cercare nella documentazione gli altri iperparametri
        ),
    integrations=[
        WandbIntegrationIn(
            project="test_mistral_api",
            run_name="pokemon_test_run",
            api_key=WANDB_API_KEY,
        ).dict()
    ],
)

In [ ]:
pprint(created_jobs)

In [ ]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

In [ ]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

In [ ]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)

In [ ]:
## inferenza
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content='What is the best French cheese?')]
)
pprint(chat_response)